library(tidyverse)  #helps wrangle data
# Use the conflicted package to manage conflicts
library(conflicted)

# Set dplyr::filter and dplyr::lag as the default choices
conflict_prefer("filter", "dplyr")
conflict_prefer("lag", "dplyr")

#=====================
# COLLECT DATA
#=====================
# # Upload Divvy datasets (csv files) here
q1_2019 <- read_csv("Divvy_Trips_2019_Q1.csv")
q1_2020 <- read_csv("Divvy_Trips_2020_Q1.csv")


#====================================================
# WRANGLE DATA AND COMBINE INTO A SINGLE FILE
#====================================================
# Compare column names each of the files
# While the names don't have to be in the same order, they DO need to match perfectly before we can use a command to join them into one file
colnames(q1_2019)
colnames(q1_2020)

# Rename columns  to make them consistent with q1_2020 (as this will be the supposed going-forward table design for Divvy)

(q1_2019 <- rename(q1_2019
                   ,ride_id = trip_id
                   ,rideable_type = bikeid
                   ,started_at = start_time
                   ,ended_at = end_time
                   ,start_station_name = from_station_name
                   ,start_station_id = from_station_id
                   ,end_station_name = to_station_name
                   ,end_station_id = to_station_id
                   ,member_casual = usertype
))

# Inspect the dataframes and look for incongruencies
str(q1_2019)
str(q1_2020)

# Convert ride_id and rideable_type to character so that they can stack correctly
q1_2019 <-  mutate(q1_2019, ride_id = as.character(ride_id)
                   ,rideable_type = as.character(rideable_type)) 

# Stack individual quarter's data frames into one big data frame
all_trips <- bind_rows(q1_2019, q1_2020)#, q3_2019)#, q4_2019, q1_2020)

# Remove lat, long, birthyear, and gender fields as this data was dropped beginning in 2020
all_trips <- all_trips %>%  
  select(-c(start_lat, start_lng, end_lat, end_lng, birthyear, gender,  "tripduration"))






#======================================================
# CLEAN UP AND ADD DATA TO PREPARE FOR ANALYSIS
#======================================================
# Inspect the new table that has been created
colnames(all_trips)  #List of column names
nrow(all_trips)  #How many rows are in data frame?
dim(all_trips)  #Dimensions of the data frame?
head(all_trips)  #See the first 6 rows of data frame.  Also tail(all_trips)
str(all_trips)  #See list of columns and data types (numeric, character, etc)
summary(all_trips)  #Statistical summary of data. Mainly for numerics

# There are a few problems to fix:
# (1) In the "member_casual" column, there are two names for members ("member" and "Subscriber") and two names for casual riders ("Customer" and "casual"). We will need to consolidate that from four to two labels.
# (2) The data can only be aggregated at the ride-level, which is too granular. We will want to add some additional columns of data -- such as day, month, year -- that provide additional opportunities to aggregate the data.
# (3) We will want to add a calculated field for length of ride since the 2020Q1 data did not have the "tripduration" column. We will add "ride_length" to the entire dataframe for consistency.
# (4) There are some rides where tripduration shows up as negative, including several hundred rides where Divvy took bikes out of circulation for Quality Control reasons. We will want to delete these rides.

# In the "member_casual" column, replace "Subscriber" with "member" and "Customer" with "casual"
# Before 2020, Divvy used different labels for these two types of riders ... we will want to make our dataframe consistent with their current nomenclature
# N.B.: "Level" is a special property of a column that is retained even if a subset does not contain any values from a specific level
# Begin by seeing how many observations fall under each usertype
table(all_trips$member_casual)

# Reassign to the desired values (we will go with the current 2020 labels)
all_trips <-  all_trips %>% 
  mutate(member_casual = recode(member_casual
                                ,"Subscriber" = "member"
                                ,"Customer" = "casual"))

# Check to make sure the proper number of observations were reassigned
table(all_trips$member_casual)

# Add columns that list the date, month, day, and year of each ride
# This will allow us to aggregate ride data for each month, day, or year ... before completing these operations we could only aggregate at the ride level

all_trips$date <- as.Date(all_trips$started_at) #The default format is yyyy-mm-dd
all_trips$month <- format(as.Date(all_trips$date), "%m")
all_trips$day <- format(as.Date(all_trips$date), "%d")
all_trips$year <- format(as.Date(all_trips$date), "%Y")
all_trips$day_of_week <- format(as.Date(all_trips$date), "%A")

# Add a "ride_length" calculation to all_trips (in seconds)
all_trips$ride_length <- difftime(all_trips$ended_at,all_trips$started_at)

# Inspect the structure of the columns
str(all_trips)

# Convert "ride_length" from Factor to numeric so we can run calculations on the data
is.factor(all_trips$ride_length)
all_trips$ride_length <- as.numeric(as.character(all_trips$ride_length))
is.numeric(all_trips$ride_length)

# Remove "bad" data
# The dataframe includes a few hundred entries when bikes were taken out of docks and checked for quality by Divvy or ride_length was negative
# We will create a new version of the dataframe (v2) since data is being removed
all_trips_v2 <- all_trips[!(all_trips$start_station_name == "HQ QR" | all_trips$ride_length<0),]

#=====================================
# DESCRIPTIVE ANALYSIS
#=====================================


summary(all_trips_v2$ride_length)

# Compare members and casual users
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = mean)
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = median)
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = max)
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = min)

# See the average ride time by each day for members vs casual users
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual + all_trips_v2$day_of_week, FUN = mean)

# Days of the week are out of order.
all_trips_v2$day_of_week <- ordered(all_trips_v2$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))

# Run the average ride time by each day for members vs casual users
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual + all_trips_v2$day_of_week, FUN = mean)

# analyze ridership data by type and weekday
all_trips_v2 %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>%  #creates weekday field using wday()
  group_by(member_casual, weekday) %>%  #groups by usertype and weekday
  summarise(number_of_rides = n()							#calculates the number of rides and average duration 
            ,average_duration = mean(ride_length)) %>% 		# calculates the average duration
  arrange(member_casual, weekday)								# sorts

# Vis - 1

# Number of rides by rider type
all_trips_v2 %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>% 
  group_by(member_casual, weekday) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length)) %>% 
  arrange(member_casual, weekday)  %>% 
  ggplot(aes(x = weekday, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge")+
  labs(title = "Number of Rides by User Type", 
       x = "Weekday", 
       y = "Number of Rides", 
       fill = "User Type") +
  theme_minimal()

#Vis - 2

# Visualization for average duration
all_trips_v2 %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>% 
  group_by(member_casual, weekday) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length)) %>% 
  arrange(member_casual, weekday)  %>% 
  ggplot(aes(x = weekday, y = average_duration, fill = member_casual)) +
  geom_col(position = "dodge")+
  labs(title = "Average Duration of User Types", 
       x = "Weekday", 
       y = "Average Duration", 
       fill = "User Type") +
  theme_minimal()

# Vis - 3

# Create a new column combining start and end station names
all_trips_v2 <- all_trips_v2 %>%
  mutate(route = paste(start_station_name, "→", end_station_name))  # Combine with arrow for clarity

# Count occurrences of each route
top_routes <- all_trips_v2 %>%
  count(route, sort = TRUE) %>%
  slice_max(n, n = 10)  # Get the top 10 most popular routes

# Filter the dataset for only the top 10 routes
df_filtered <- all_trips_v2 %>%
  filter(route %in% top_routes$route)

# Aggregate count by route and user type
df_aggregated <- df_filtered %>%
  count(route, member_casual)

# Create the plot
ggplot(df_aggregated, aes(x = reorder(route, n), y = n, fill = member_casual)) +
  geom_col() +
  coord_flip() +  # Flip for better readability
  labs(title = "Membership Distribution by Top 10 Routes", 
       x = "Route (Start → End)", 
       y = "Total Rides", 
       fill = "User Type") +
  theme_minimal()

# Vis - 4

library(lubridate)

# Convert start & end times to hours and extract day of the week
df_time <- all_trips_v2 %>%
  mutate(
    day_of_week = wday(started_at, label = TRUE),  # Extract weekday name
    start_hour = hour(started_at),                # Extract start hour
    end_hour = hour(ended_at)                     # Extract end hour
  ) %>%
  group_by(member_casual, day_of_week) %>%
  summarise(
    avg_start_time = mean(start_hour, na.rm = TRUE),  # Average start time per day
    avg_end_time = mean(end_hour, na.rm = TRUE),      # Average end time per day
    .groups = "drop"
  ) %>%
  pivot_longer(cols = c(avg_start_time, avg_end_time), 
               names_to = "time_type", values_to = "average_hour")

# Plot
ggplot(df_time, aes(x = day_of_week, y = average_hour, color = time_type, group = time_type)) +
  geom_line(size = 1) +
  facet_wrap(~ member_casual) +  # Separate graphs for casual & members
  labs(title = "Average Start and End Times Over a Week",
       x = "Day of the Week", y = "Average Hour", color = "Time Type") +
  theme_minimal()

# Vis 5

summary(q1_2019)  # Shows NAs in each column
colSums(is.na(q1_2019))  # Counts NAs per column

df_1 <- q1_2019 %>%
  select(gender, birthyear, member_casual) %>%  # Select relevant columns
  mutate(birthyear = as.numeric(birthyear)) %>%
  drop_na() %>%  # Remove NA values
  mutate(age = 2019 - birthyear) %>%
  drop_na(birthyear, gender) %>%  # Remove rows with missing birthyear or gender
  mutate(age = 2019 - birthyear)


# Plot Violin Chart
ggplot(df_1, aes(x = gender, y = age, fill = gender)) +
  geom_violin() +
  scale_y_continuous(breaks = seq(0, max(df_1$age), by = 10)) +  # Set Y-axis increments to 10
  labs(title = "Age Distribution by Gender",
       x = "Age Group",
       y = "Age",
       fill = "Gender") +
  theme_classic()



#=================================================
# EXPORT SUMMARY FILE FOR FURTHER ANALYSIS
#=================================================
# Create a csv file
counts <- aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual + all_trips_v2$day_of_week, FUN = mean)
write.csv(counts, file = 'avg_ride_length.csv')